In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/AI_ML_SEM_2

/content/drive/MyDrive/AI_ML_SEM_2


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv2d_drop = nn.Dropout()
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 14)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2d_drop(self.conv1(x))))
        x = self.pool(F.relu(self.conv2(x)))
        # x = self.pool(F.relu(self.conv2d_drop(self.conv2(x))))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
class SoftmaxClassifier(nn.Module):
    def __init__(self):
        super(SoftmaxClassifier, self).__init__()
        self.fc = nn.Linear(28 * 28 * 3, 14, bias=False)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data.normal_().mul_(0.005)

    def forward(self, x):
        x = x.view(-1, 28 * 28 * 3)
        x = self.fc(x)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
 
import argparse
import os
import shutil
import numpy as np
import time
import utils
import models
 
from logger import Logger

In [ ]:
parser = argparse.ArgumentParser(description='Training and Evaluation')


wrk = 4 
#epochs
ep = 90 
#start-epoch
sep = 0
#batch-size
bs = 256
#learning-rate
lr_1 = 0.1
#momentum
mome = 0.9 
#weight-decay
wd = 1e-4
#print-freq
p_1 = 10
#resume
res = None
#evaluate
evaluate_1 = True

In [ ]:
def main():
    global args 
    args = parser.parse_args()

    debug = 1 

    
    traindir = os.path.join(args.data, './TrainData/Real_Train/ChallengeFree')
    testdir = os.path.join(args.data, './ChallengeFree_Test')

    train_dataset = utils.CURETSRDataset(traindir, transforms.Compose([
        transforms.Resize([28, 28]), transforms.ToTensor(), utils.l2normalize, utils.standardization]))
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True,
                                               num_workers=wrk, pin_memory=True)

    test_dataset = utils.CURETSRDataset(testdir, transforms.Compose([
        transforms.Resize([28, 28]), transforms.ToTensor(), utils.l2normalize, utils.standardization]))
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=bs, shuffle=False,
                                              num_workers=wrk, pin_memory=True)

    model = models.Net()
    model = torch.nn.DataParallel(model).cuda()
    print("=> creating model %s " % model.__class__.__name__)

    savedir = 'CNN_iter'
    checkpointdir = os.path.join('./checkpoints', savedir)

    if not debug:
        os.mkdir(checkpointdir)
        print('log directory: %s' % os.path.join('./logs', savedir))
        print('checkpoints directory: %s' % checkpointdir)

    # Set the logger
    if not debug:
        logger = Logger(os.path.join('./logs/', savedir))

    # define loss function (criterion) and optimizer
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr_1,mome,
                                weight_decay=wd)

    # optionally resume from a checkpoint
    if res:
        if os.path.isfile(res):
            print("=> loading checkpoint '{}'".format(res))
            checkpoint = torch.load(res)
            sep = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            print("=> loaded checkpoint '{}' (epoch {}, best_prec1 @ Source {})"
                  .format(res, checkpoint['epoch'], best_prec1))
        else:
            print("=> no checkpoint found at '{}'".format(res))

    if evaluate_1:
        evaluate(test_loader, model, criterion)
        return

    cudnn.benchmark = True

    timestart = time.time()
    best_prec1 = 0

    for epoch in range(sep, ep):
        adjust_learning_rate(optimizer, epoch)

        # train for one epoch
        print('\n*** Start Training *** \n')
        train(train_loader, model, criterion, optimizer, epoch)

        # evaluate on validation set
        print('\n*** Start Testing *** \n')
        test_loss, test_prec1, _ = evaluate(test_loader, model, criterion)

        info = {
            'Testing loss': test_loss,
            'Testing Accuracy': test_prec1
        }

        # remember best prec@1 and save checkpoint
        is_best = test_prec1 > best_prec1
        best_prec1 = max(test_prec1, best_prec1)

        if is_best:
            best_epoch = epoch + 1

        if not debug:
            for tag, value in info.items():
                logger.scalar_summary(tag, value, epoch+1)

                save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': model.state_dict(),
                    'best_prec1': best_prec1,
                    'last_prec1': test_prec1,
                    'optimizer': optimizer.state_dict()}, is_best, checkpointdir)


    print('Best epoch: ', best_epoch)
    print('Total processing time: %.4f' % (time.time() - timestart))


def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        data_time.update(time.time() - end)

        target = target.cuda()
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)
        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))

        losses.update(loss.data, input.size(0))  # input.size(0): Batch size
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % p_1 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5))
    
    return losses.avg, top1.avg, top5.avg



In [ ]:
def evaluate(test_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(test_loader):
        target = target.cuda()
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data, input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % p_1 == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, len(test_loader), batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))

    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return losses.avg, top1.avg, top5.avg

In [ ]:
def save_checkpoint(state, is_best, checkpointdir):
    fullpath = os.path.join(checkpointdir, 'checkpoint.pth.tar')
    fullpath_best = os.path.join(checkpointdir, 'model_best.pth.tar')
    torch.save(state, fullpath)

    if is_best:
        shutil.copyfile(fullpath, fullpath_best)


In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = lr_1 * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    res = []

    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


if __name__ == '__main__':
    main()

In [ ]:
!python train.py --lr 0.001 ./TrainData


2021-07-28 17:47:38.247024: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
<class 'str'>
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
=> creating model DataParallel 

*** Start Training *** 

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  retu